## 1. Show me the Credit

- I am completing a (closed) kaggle competetion - show me the credit
- The competetion provided two data sets: a test set and and a training set




### 1.1 Importing packages
- I have imported packages I am **definately** using: Pandas(and numpy) to read and examine the data; matplotlib and seaborn to plot
- I also know that i'll be using knn to impute values and logistical regression to predict classes 

In [51]:
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import numpy as np


In [52]:

user_cols = ['dlq' , 'credit_utilization', 'age', 'past_30', 'debtratio', 
             'income', 'credit_lines', 'past_90', 'home_lines',
             'past_60', 'dep_count']
             

In the code block above, I created columns names to replace existing ones. 
###Variable Name:

SeriousDlqin2yrs

RevolvingUtilizationOfUnsecuredLines

age

NumberOfTime30-59DaysPastDueNotWorse

DebtRatio

MonthlyIncome

NumberOfOpenCreditLinesAndLoans

NumberOfTimes90DaysLate

NumberRealEstateLoansOrLines

NumberOfTime60-89DaysPastDueNotWorse

NumberOfDependents

###Description:

Person experienced 90 days past due delinquency or worse 

Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits

Age of borrower in years

Number of times borrower has been 30-59 days past due but no worse in the last 2 years.

Monthly debt payments, alimony,living costs divided by monthy gross income

Monthly income

Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)

Number of times borrower has been 90 days or more past due.

Number of mortgage and real estate loans including home equity lines of credit

Number of times borrower has been 60-89 days past due but no worse in the last 2 years.

Number of dependents in family excluding themselves (spouse, children etc.)


In [53]:
training = pd.read_table('cs-training.csv', sep=',', header=False,\
 names=user_cols)
#read file, changed index, changed names
 
test = pd.read_table('cs-test.csv', sep = ',', header=False,\
names=user_cols)


In [54]:
training.debtratio.describe()

count    150000.000000
mean        353.005076
std        2037.818523
min           0.000000
25%           0.175074
50%           0.366508
75%           0.868254
max      329664.000000
Name: debtratio, dtype: float64

In [6]:
import numpy as np

def take_log(n):
    return np.log(n)

training['lnCredit'] = training.credit_utilization.replace(to_replace=0, value = 0.1).apply(take_log)
test['lnCredit'] = test.credit_utilization.replace(to_replace=0, value = 0.1).apply(take_log)

training['lnDR'] = training.debtratio.replace(to_replace=0, value = 0.1).apply(take_log)
test['lnDR'] = test.debtratio.replace(to_replace=0, value = 0.1).apply(take_log)

training['ln30'] = training.past_30.replace(to_replace=0, value = 0.1).apply(take_log)
training['ln60'] = training.past_60.replace(to_replace=0, value = 0.1).apply(take_log)
training['ln90'] = training.past_90.replace(to_replace=0, value = 0.1).apply(take_log)

test['ln30'] = test.past_30.replace(to_replace=0, value = 0.1).apply(take_log)
test['ln60'] = test.past_60.replace(to_replace=0, value = 0.1).apply(take_log)
test['ln90'] = test.past_90.replace(to_replace=0, value = 0.1).apply(take_log)


####How each feature relates to the response

####Observations: 

    1) There are a lot of missing values, in income and dependent columns
    2) In the following lines of code, I will impute dependent count using sk-learn's imputer, and income using KNN Regressor
 

In [7]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=1)
training['dep_count'] = imp.fit_transform(training.dep_count).T


In [8]:
training_imp = training.dropna()  
training_imp['lnIncome'] = training.income.replace(to_replace=0, value = 0.1).apply(take_log)

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [9]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn import neighbors
yi = training_imp.income
Xi = training_imp[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR', 'lnIncome']]

In [13]:
Xi_train, Xi_test, yi_train, yi_test = train_test_split(Xi, yi, \
                                                        test_size=0.4, random_state=4)
from sklearn.ensemble import RandomForestRegressor


rfreg = RandomForestRegressor(n_estimators=150, max_features=8, oob_score=True, random_state=1)

rfreg.fit(Xi_train, yi_train)
yi_pred = rfreg.predict(Xi_test)

print np.sqrt(metrics.mean_squared_error(yi_test, yi_pred)) #1519 baggedRegressor, vs 14454 with KNN Regressor

1553.64369829


In [14]:
training_null = training[training.income.isnull() == True]

In [15]:
training_null.shape

(29731, 16)

In [16]:
y_null = training_null.income
X_null = training_null[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR']]

In [17]:
Xi = training_imp[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR']]


rfreg.fit(Xi, yi)

y_null_pred = rfreg.predict(X_null)

In [18]:
y_null = y_null_pred 


In [19]:
training_null.income = y_null

//anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


###The code above did the following:
    - Successfully imputed missing dependent counts using Sci-kit learn's impute function(although, that might have been redundant since it imputed 0 - the most frequent value)
    - Successfully imputed income using KNN Regressor.
    
    
###The objective of the code following this is to:
    - Run and evaluate a Logistical Regression on the now complete training set
    - impute missing values in the testing set
    - run LR in the training set. 
    

In [20]:
training_1 = pd.concat([training_imp, training_null])

In [21]:
logreg = LogisticRegression(C=1e9)

training_1['lnIncome'] = training_1.income.replace(to_replace=0, value = 0.1).apply(take_log)

X = training_1[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR', 'lnIncome']]
y = training_1.dlq

In [24]:
from sklearn.cross_validation import cross_val_score
cross_val_score(logreg, X, y, cv=10, scoring='roc_auc').mean()

#vanilla .695
# better income: .849

0.84927542723025784

###Imputing missing values in the testing set

In [22]:
test.isnull().sum()

dlq                   101503
credit_utilization         0
age                        0
past_30                    0
debtratio                  0
income                 20103
credit_lines               0
past_90                    0
home_lines                 0
past_60                    0
dep_count               2626
lnCredit                   0
lnDR                       0
ln30                       0
ln60                       0
ln90                       0
dtype: int64

In [23]:
test['dep_count'] = imp.fit_transform(test.dep_count).T

imputed dep counts, need to impute income

In [24]:
test_null = test[test.income.isnull() == True]


In [25]:
test.head()

,dlq,credit_utilization,age,past_30,debtratio,income,credit_lines,past_90,home_lines,past_60,dep_count,lnCredit,lnDR,ln30,ln60,ln90
1,NaN,0.885519,43,0,0.177513,5700,4,0,0,0,0,-1.215813e-01,-1.728713,-2.302585,-2.302585,-2.302585
2,NaN,0.463295,57,0,0.527237,9141,15,0,4,0,2,-7.693907e-01,-0.640105,-2.302585,-2.302585,-2.302585
3,NaN,0.043275,59,0,0.687648,5083,12,0,1,0,2,-3.140179e+00,-0.374479,-2.302585,-2.302585,-2.302585
4,NaN,0.280308,38,1,0.925961,3200,7,0,2,0,0,-1.271865e+00,-0.076924,0.000000,-2.302585,-2.302585
5,NaN,1.000000,27,0,0.019917,3865,4,0,0,0,1,-1.000000e-07,-3.916170,-2.302585,-2.302585,-2.302585


In [26]:
test_1 = test.loc[:, 'credit_utilization' : 'ln90']

In [27]:
test_1.isnull().sum()

credit_utilization        0
age                       0
past_30                   0
debtratio                 0
income                20103
credit_lines              0
past_90                   0
home_lines                0
past_60                   0
dep_count                 0
lnCredit                  0
lnDR                      0
ln30                      0
ln60                      0
ln90                      0
dtype: int64

In [28]:
test_imp = test_1.dropna()

In [29]:
test_imp.shape #successfully seperated all null incomes, to impute tthem

(81400, 15)

In [30]:
test_imp['lnIncome'] = test.income.replace(to_replace=0, value = 0.1).apply(take_log)

yj = test_imp.income
Xj = test_imp[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR']]



Xj_null = test_null[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR']]



rfreg.fit(Xj, yj)

y_null_pred = rfreg.predict(Xj_null)


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
test_null.income = y_null_pred

In [32]:
test_1 = pd.concat([test_imp, test_null])

In [33]:
test_1.shape # shape looks correct

(101503, 17)

In [34]:
test_1['lnIncome'] = test_1.income.replace(to_replace=0, value = 0.1).apply(take_log)

In [36]:
from sklearn.ensemble import RandomForestClassifier

rfclass = RandomForestClassifier(n_estimators=150, max_features=8, oob_score=True, random_state=1)

In [46]:
refclass = RandomForestClassifier(n_estimators = 100, max_features=8, oob_score=True, random_state=1)
refclass.fit(X, y)

Xii = test_1[['credit_utilization', 'age', 'debtratio', 
             'past_90', 'home_lines',
             'past_60', 'dep_count', 'ln60', 'ln30', 'ln90', 'lnCredit', 'lnDR', 'lnIncome']]

first_prediction = refclass.predict_proba(Xii)[:, 1]

In [45]:
sub = pd.DataFrame({'id':test_1.index, 'Probability':first_prediction}).set_index('id')
sub.to_csv('sub_rf.csv')

In [47]:
sub_test = pd.DataFrame({
                        'income':test_1.income, })

In [49]:
sub_train = pd.DataFrame({'income':training_1.income, })

sub_train.to_csv('train_clean_1.csv')

In [50]:
sub_test.to_csv('test_clean_1.csv')

In [2]:
sub_train.head()

NameError: name 'sub_train' is not defined